In [2]:
import gradio as gr
import uuid
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, ValidationError, Field
from crewai.flow import Flow, listen, start, persist, router, or_
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
import os
from langchain_openai import ChatOpenAI
from llm import togetherai_llm
from datetime import datetime
import json
import re
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool,
)
# --- Models Definitions (Unchanged) ---
class ContentInfo(BaseModel):
    topic: Optional[str] = Field(default=None, description="Chủ đề của nội dung")
    communication_goal: Optional[str] = Field(default=None, description="Mục tiêu truyền thông")
    target_audience: Optional[str] = Field(default=None, description="Đối tượng mục tiêu")
    brand_information: Optional[str] = Field(default=None, description="Thông tin thương hiệu")
    style_and_tone: Optional[str] = Field(default=None, description="Phong cách và giọng điệu")

class ContentIdea(BaseModel):
    title: str = Field(description="Tiêu đề ý tưởng")
    description: str = Field(description="Mô tả ngắn gọn về ý tưởng")
    content_type: str = Field(description="Loại nội dung (bài viết, hình ảnh, video, v.v.)")
    reason: str = Field(description="Lý do ý tưởng phù hợp với mục tiêu và đối tượng")
# Create a wrapper class for the List[ContentIdea]
class ContentIdeaList(BaseModel):
    ideas: List[ContentIdea] = Field(default_factory=list)
    
class ContentResearch(BaseModel):
    keywords: List[str] = Field(description="Các từ khóa liên quan")
    competitor_analysis: str = Field(description="Phân tích đối thủ cạnh tranh")
    trending_topics: List[str] = Field(description="Các chủ đề đang thịnh hành")
    insights: str = Field(description="Hiểu biết và phát hiện từ nghiên cứu")

class ContentDraft(BaseModel):
    title: str = Field(description="Tiêu đề nội dung")
    body: str = Field(description="Nội dung chính")
    hashtags: List[str] = Field(description="Hashtags liên quan")
    call_to_action: str = Field(description="Kêu gọi hành động")

class ResearchSynthesis(BaseModel):
    key_insights: List[str] = Field(description="Các insight chính từ dữ liệu")
    unique_angle: str = Field(description="Góc nhìn độc đáo để tiếp cận chủ đề")
    key_elements: List[str] = Field(description="Các yếu tố quan trọng cần đưa vào nội dung")
    recommendations: List[str] = Field(description="Đề xuất cụ thể cho quá trình viết nội dung")

class ChatState(BaseModel):
    message: Optional[str] = Field(default=None, description="Nội dung tin nhắn hiện tại")
    history: Optional[str] = Field(default=None, description="Lịch sử chat")
    content_info: Dict[str, Any] = Field(default_factory=dict)
    content_ideas: List[ContentIdea] = Field(default_factory=list, description="Ý tưởng nội dung")
    selected_idea_index: Optional[int] = Field(default=None, description="Chỉ số ý tưởng được chọn")
    research_data: Optional[ContentResearch] = Field(default=None, description="Dữ liệu nghiên cứu")
    synthesis_data: Optional[ResearchSynthesis] = Field(default=None, description="Dữ liệu tổng hợp nghiên cứu")
    content_draft: Optional[ContentDraft] = Field(default=None, description="Bản thảo nội dung")
    final_content: Optional[str] = Field(default=None, description="Nội dung cuối cùng")
    current_step: str = Field(default="start", description="Bước hiện tại trong quy trình")
    is_complete: bool = Field(default=False, description="Trạng thái hoàn thành")
    def model_dump(self, *args, **kwargs):
        # Chuyển đổi content_info thành dictionary trước khi dump
        data = super().model_dump(*args, **kwargs)
        if isinstance(self.content_info, ContentInfo):
            data["content_info"] = self.content_info.model_dump()
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> "ChatState":
        # Khôi phục lại content_info thành một object ContentInfo khi load
        if "content_info" in data and isinstance(data["content_info"], dict):
            data["content_info"] = ContentInfo(**data["content_info"])
        return cls(**data)
    def json_serializable(self):
        data = self.model_dump()
        data["content_info"] = self.content_info.model_dump()
        if self.research_data:
            data["research_data"] = self.research_data.model_dump()
        if self.synthesis_data:
            data["synthesis_data"] = self.synthesis_data.model_dump()
        if self.content_draft:
            data["content_draft"] = self.content_draft.model_dump()
        return data

# --- Optimized CrewAI Implementation ---
@CrewBase
class ContentCreationCrew:
    """Unified Content Creation crew handling the entire process with a single crew"""
    
    # Config files
    agents_config = "config/agents.yaml"
    tasks_config = "config/tasks.yaml"

    @agent
    def content_manager(self) -> Agent:
        return Agent(
            config=self.agents_config["content_manager"],
            llm=togetherai_llm,
            verbose=True,
            memory=True
        )
    
    @agent
    def content_researcher(self) -> Agent:
        return Agent(
            config=self.agents_config["content_researcher"],
            llm=togetherai_llm,
            verbose=True,
            memory=True,
            tools = [SerperDevTool()]
        )
    @agent
    def research_synthesizer(self) -> Agent:
        return Agent(
            config=self.agents_config["content_researcher"],
            llm=togetherai_llm,
            verbose=True,
            memory=True
        )
    @agent
    def content_writer(self) -> Agent:
        return Agent(
            config=self.agents_config["content_writer"],
            llm=togetherai_llm,
            verbose=True,
            memory=True
        )
    
    @agent
    def content_reviewer(self) -> Agent:
        return Agent(
            config=self.agents_config["content_reviewer"],
            llm=togetherai_llm,
            verbose=True,
            memory=True
        )

    @task
    def ideation_task(self) -> Task:
        return Task(
            config=self.tasks_config["content_ideation_task"],
            agent=self.content_manager(),
            output_pydantic=ContentIdeaList
        )
    
    @task
    def research_task(self) -> Task:
        return Task(
            config=self.tasks_config["research_task"],
            agent=self.content_researcher(),
            output_pydantic=ContentResearch,
            context=[self.ideation_task]  # Use the ideation task as context
        )
    
    @task
    def synthesis_task(self) -> Task:
        return Task(
            config=self.tasks_config["research_synthesis_task"],
            agent=self.research_synthesizer(),
            output_pydantic=ResearchSynthesis,
            context=[self.research_task]  # Use the research task as context
        )
    
    @task
    def writing_task(self) -> Task:
        return Task(
            config=self.tasks_config["content_writing_task"],
            agent=self.content_writer(),
            output_pydantic=ContentDraft,
            context=[self.ideation_task,  self.synthesis_task]  # Use all previous tasks as context
        )
    
    
    @task
    def review_task(self) -> Task:
        return Task(
            config=self.tasks_config["content_review_task"],
            agent=self.content_reviewer(),
            context=[self.writing_task, self.ideation_task]  # Use writing and research as context
        )
        
    @crew
    def crew(self) -> Crew:
        """Creates the Content Creation Crew with all agents and tasks"""
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,  # We can still use sequential process but leverage the task context
            verbose=True,
            
        )

@persist()
class ContentChatFlow(Flow[ChatState]):
    def __init__(self, persistence=None):
        super().__init__(persistence=persistence)
        # Initialize single unified crew
        self.content_crew = ContentCreationCrew().crew()
        self.current_date = datetime.now().strftime("%Y-%m-%d-%H-%M")

    @start()
    def start_flow(self):
        """Start the processing flow, display current status"""
        print("Bắt đầu quy trình tạo nội dung")
        print("Thông tin nội dung:", self.state.content_info)
        self.state.current_step = "ideation"
        response = "Tôi sẽ giúp bạn tạo nội dung dựa trên thông tin bạn đã cung cấp. Đang bắt đầu quá trình tạo ý tưởng..."
        return response
    
    @listen(start_flow)
    def generate_content_ideas(self):
        """Generate content ideas based on user information"""
        print("Đang tạo ý tưởng nội dung...")
        
        inputs = {
            "message": self.state.message,
            "content_info": self.state.content_info.model_dump(),
            "history": self.state.history,
            "current_date": self.current_date,
            "task": "ideation"  # Specify which task to run
        }
        
        result = self.content_crew.tasks[0].execute(inputs=inputs)  # Execute ideation task
        
        if hasattr(result, 'pydantic'):
            self.state.content_ideas = result.pydantic
            print(f"Đã tạo {len(self.state.content_ideas)} ý tưởng nội dung")
            
            # Prepare response for user
            ideas_text = ""
            for i, idea in enumerate(self.state.content_ideas):
                ideas_text += f"\n\n{i+1}. **{idea.title}**\n"
                ideas_text += f"   - Loại nội dung: {idea.content_type}\n"
                ideas_text += f"   - Mô tả: {idea.description}\n"
                ideas_text += f"   - Lý do: {idea.reason}"
            
            response = f"Tôi đã tạo {len(self.state.content_ideas)} ý tưởng nội dung dựa trên thông tin bạn cung cấp:{ideas_text}\n\nBạn muốn chọn ý tưởng nào để tiếp tục? (Vui lòng nhập số thứ tự từ 1-{len(self.state.content_ideas)})"
            
            self.state.current_step = "select_idea"
            return response
        else:
            print("Không thể tạo ý tưởng nội dung")
            return "Rất tiếc, tôi không thể tạo ý tưởng nội dung. Vui lòng cung cấp thêm thông tin và thử lại."
    
    @router(generate_content_ideas)
    def process_user_selection(self):
        """Process user's selection of content idea"""
        if self.state.current_step != "select_idea":
            return self.state.current_step
            
        try:
            # Try to parse number from user message
            user_message = self.state.message
            
            # Find numbers in message
            numbers = re.findall(r'\d+', user_message)
            
            if numbers:
                selected_index = int(numbers[0]) - 1  # Convert from 1-based to 0-based index
                
                if 0 <= selected_index < len(self.state.content_ideas):
                    self.state.selected_idea_index = selected_index
                    selected_idea = self.state.content_ideas[selected_index]
                    
                    response = f"Bạn đã chọn ý tưởng: **{selected_idea.title}**. Tôi sẽ bắt đầu nghiên cứu để phát triển nội dung này."
                    print(f"Người dùng đã chọn ý tưởng: {selected_idea.title}")
                    
                    self.state.current_step = "research"
                    return "research"
                else:
                    print(f"Lựa chọn không hợp lệ: {selected_index+1}, yêu cầu từ 1-{len(self.state.content_ideas)}")
                    return "invalid_selection"
            else:
                print("Không tìm thấy số trong tin nhắn của người dùng")
                return "invalid_selection"
                
        except Exception as e:
            print(f"Lỗi khi xử lý lựa chọn: {e}")
            return "invalid_selection"
    
    @listen("invalid_selection")
    def handle_invalid_selection(self):
        """Handle invalid user selection"""
        response = f"Vui lòng chọn một số từ 1 đến {len(self.state.content_ideas)} để tiếp tục."
        self.state.current_step = "select_idea"  # Stay at idea selection step
        return response
    
    @listen("research")
    def conduct_research(self):
        """Conduct research for the selected idea"""
        print("Đang tiến hành nghiên cứu...")
        
        selected_idea = self.state.content_ideas[self.state.selected_idea_index]
        
        inputs = {
            "content_info": self.state.content_info.model_dump(),
            "selected_idea": selected_idea.model_dump(),
            "current_date": self.current_date,
            "task": "research"  # Specify which task to run
        }
        
        result = self.content_crew.tasks[1].execute(inputs=inputs)  # Execute research task
        
        if hasattr(result, 'pydantic'):
            self.state.research_data = result.pydantic
            print("Nghiên cứu hoàn tất")
            
            # Now run synthesis task automatically without user interaction
            self.state.current_step = "synthesis"
            return self.synthesize_research()
        else:
            print("Không thể hoàn thành nghiên cứu")
            return "Rất tiếc, tôi không thể hoàn thành nghiên cứu. Vui lòng thử lại sau."
    
    def synthesize_research(self):
        """Synthesize research findings - now an internal method that's called automatically"""
        print("Đang tổng hợp nghiên cứu...")
        
        selected_idea = self.state.content_ideas[self.state.selected_idea_index]
        
        inputs = {
            "content_info": self.state.content_info.model_dump(),
            "selected_idea": selected_idea.model_dump(),
            "research_data": self.state.research_data.model_dump(),
            "current_date": self.current_date,
            "task": "synthesis"  # Specify which task to run
        }
        
        result = self.content_crew.tasks[2].execute(inputs=inputs)  # Execute synthesis task
        
        if hasattr(result, 'pydantic'):
            self.state.synthesis_data = result.pydantic
            print("Tổng hợp nghiên cứu hoàn tất")
            
            keywords_text = ", ".join(self.state.research_data.keywords)
            trending_topics_text = ", ".join(self.state.research_data.trending_topics)
            insights_text = ", ".join(self.state.synthesis_data.key_insights)
            
            response = f"Tôi đã hoàn thành nghiên cứu cho ý tưởng **{selected_idea.title}**.\n\n"
            response += f"**Từ khóa quan trọng:** {keywords_text}\n\n"
            response += f"**Chủ đề đang thịnh hành:** {trending_topics_text}\n\n"
            response += f"**Phân tích đối thủ:** {self.state.research_data.competitor_analysis}\n\n"
            response += f"**Insight chính:** {insights_text}\n\n"
            response += f"**Góc tiếp cận độc đáo:** {self.state.synthesis_data.unique_angle}\n\n"
            response += "Bây giờ tôi sẽ bắt đầu viết nội dung dựa trên những thông tin này. Vui lòng đợi một chút..."
            
            self.state.current_step = "writing"
            return response
        else:
            print("Không thể tổng hợp nghiên cứu")
            # Continue to writing anyway, synthesis is optional
            self.state.current_step = "writing"
            return self.write_content()
    
    @listen("writing")
    def write_content(self):
        """Write content based on idea and research"""
        print("Đang viết nội dung...")
        
        selected_idea = self.state.content_ideas[self.state.selected_idea_index]
        
        inputs = {
            "content_info": self.state.content_info.model_dump(),
            "selected_idea": selected_idea.model_dump(),
            "research_data": self.state.research_data.model_dump(),
            "synthesis_data": self.state.synthesis_data.model_dump() if self.state.synthesis_data else None,
            "current_date": self.current_date,
            "task": "writing"  # Specify which task to run
        }
        
        result = self.content_crew.tasks[3].execute(inputs=inputs)  # Execute writing task
        
        if hasattr(result, 'pydantic'):
            self.state.content_draft = result.pydantic
            print("Bản thảo nội dung đã được tạo")
            
            response = f"Tôi đã hoàn thành bản thảo nội dung cho **{selected_idea.title}**.\n\n"
            response += f"**Tiêu đề:** {self.state.content_draft.title}\n\n"
            response += f"**Nội dung:**\n{self.state.content_draft.body}\n\n"
            response += f"**Hashtags:** {', '.join(self.state.content_draft.hashtags)}\n\n"
            response += f"**Kêu gọi hành động:** {self.state.content_draft.call_to_action}\n\n"
            response += "Bạn muốn tôi điều chỉnh nội dung này không? Nếu có, hãy cho tôi biết những thay đổi cụ thể. Nếu bạn hài lòng, hãy gõ 'OK' để tôi hoàn thiện nó."
            
            self.state.current_step = "review"
            return response
        else:
            print("Không thể tạo bản thảo nội dung")
            return "Rất tiếc, tôi không thể tạo bản thảo nội dung. Vui lòng thử lại sau."
    
    @router(write_content)
    def process_review_feedback(self):
        """Process user feedback on draft"""
        if self.state.current_step != "review":
            return self.state.current_step
            
        user_message = self.state.message.lower()
        
        if user_message in ["ok", "okay", "yes", "đồng ý", "tốt", "được"]:
            print("Người dùng chấp nhận bản thảo")
            self.state.current_step = "finalize"
            return "finalize"
        else:
            print("Người dùng yêu cầu chỉnh sửa")
            self.state.current_step = "revise"
            return "revise"
    
    @listen("revise")
    def revise_content(self):
        """Revise content based on user feedback"""
        print("Đang chỉnh sửa nội dung...")
        
        inputs = {
            "content_info": self.state.content_info.model_dump(),
            "content_draft": self.state.content_draft.model_dump(),
            "feedback": self.state.message,
            "current_date": self.current_date,
            "task": "revision"  # Specify which task to run
        }
        
        result = self.content_crew.tasks[4].execute(inputs=inputs)  # Execute revision task
        
        if hasattr(result, 'pydantic'):
            self.state.content_draft = result.pydantic
            print("Bản thảo nội dung đã được chỉnh sửa")
            
            response = f"Tôi đã chỉnh sửa bản thảo theo ý kiến của bạn.\n\n"
            response += f"**Tiêu đề:** {self.state.content_draft.title}\n\n"
            response += f"**Nội dung:**\n{self.state.content_draft.body}\n\n"
            response += f"**Hashtags:** {', '.join(self.state.content_draft.hashtags)}\n\n"
            response += f"**Kêu gọi hành động:** {self.state.content_draft.call_to_action}\n\n"
            response += "Bạn có hài lòng với phiên bản này không? Nếu có, hãy gõ 'OK'. Nếu không, hãy tiếp tục cung cấp phản hồi để tôi chỉnh sửa thêm."
            
            self.state.current_step = "review"
            return response
        else:
            print("Không thể chỉnh sửa bản thảo")
            return "Rất tiếc, tôi không thể chỉnh sửa bản thảo. Vui lòng thử lại sau."
    
    @listen("finalize")
    def finalize_content(self):
        """Finalize the content"""
        print("Đang hoàn thiện nội dung...")
        
        inputs = {
            "content_info": self.state.content_info.model_dump(),
            "content_draft": self.state.content_draft.model_dump(),
            "research_data": self.state.research_data.model_dump(),
            "synthesis_data": self.state.synthesis_data.model_dump() if self.state.synthesis_data else None,
            "current_date": self.current_date,
            "task": "review"  # Specify which task to run
        }
        
        result = self.content_crew.tasks[5].execute(inputs=inputs)  # Execute review task
        
        self.state.final_content = result.raw
        print("Nội dung cuối cùng đã được hoàn thiện")
        
        response = f"Tôi đã hoàn thiện nội dung cuối cùng.\n\n{self.state.final_content}\n\n"
        response += "Cảm ơn bạn đã sử dụng dịch vụ tạo nội dung của chúng tôi! Bạn có thể lưu nội dung này hoặc bắt đầu một dự án mới."
        
        self.state.current_step = "complete"
        self.state.is_complete = True
        return response

def kickoff():
    """
    Run the flow.
    """
    lead_score_flow = ContentChatFlow()
    lead_score_flow.kickoff()


def plot():
    """
    Plot the flow.
    """
    lead_score_flow = ContentChatFlow()
    lead_score_flow.plot()


if __name__ == "__main__":
    kickoff()
    plot()

/home/dattq/miniconda3/envs/chatbot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: source code not available

In [7]:
from pydantic import BaseModel, Field
from typing import List, Optional
import uuid
import json

# Định nghĩa mô hình ứng viên
class Candidate(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid.uuid4()))  # Tự động sinh ID
    name: str = Field(..., min_length=3, max_length=50)
    email: str
    bio: Optional[str] = None
    skills: str

# Định nghĩa trạng thái lưu danh sách ứng viên
class LeadScoreState(BaseModel):
    candidates: List[Candidate] = []
    
    # 1️⃣ Thêm ứng viên mới
    def add_candidate(self, name: str, email: str, bio: Optional[str], skills: str):
        candidate = Candidate(name=name, email=email, bio=bio, skills=skills)
        self.candidates.append(candidate)
        return candidate

    # 2️⃣ Lấy danh sách ứng viên
    def list_candidates(self):
        return self.candidates

    # 3️⃣ Xóa ứng viên theo ID
    def remove_candidate(self, candidate_id: str):
        self.candidates = [c for c in self.candidates if c.id != candidate_id]

    # 4️⃣ Xuất dữ liệu ra JSON
    def export_to_json(self):
        return self.json(indent=4)

# Khởi tạo danh sách ứng viên
state = LeadScoreState()


In [8]:
candidate1 = state.add_candidate(
    name="Nguyễn Văn A",
    email="nguyenvana@example.com",
    bio="Lập trình viên Python với 5 năm kinh nghiệm",
    skills="Python, Django, Machine Learning"
)

candidate2 = state.add_candidate(
    name="Trần Thị B",
    email="tranthib@example.com",
    bio="Chuyên gia AI",
    skills="AI, NLP, Deep Learning"
)


In [ ]:
class ContentInfo(BaseModel):
    topic: Optional[str] = Field(default=None, description="Chủ đề của nội dung")
    communication_goal: Optional[str] = Field(default=None, description="Mục tiêu truyền thông")
    target_audience: Optional[str] = Field(default=None, description="Đối tượng mục tiêu")
    brand_information: Optional[str] = Field(default=None, description="Thông tin thương hiệu")
    style_and_tone: Optional[str] = Field(default=None, description="Phong cách và giọng điệu")
# Define ContentIdea model
class ContentIdea(BaseModel):
    title: Optional[str] = Field(description="Tiêu đề ý tưởng")
    description: Optional[str] = Field(description="Mô tả ngắn gọn về ý tưởng")
    content_type: Optional[str] = Field(description="Loại nội dung (bài viết, hình ảnh, video, v.v.)")
    reason: Optional[str] = Field(description="Lý do ý tưởng phù hợp với mục tiêu và đối tượng")

# Create a wrapper class for the List[ContentIdea]
class ContentIdeaList(BaseModel):
    ideas: List[ContentIdea] = Field(default_factory=list)

class ChatState(BaseModel):
    id: str = ""
    message: Optional[str] =  Field(default=None, description="Nội dung tin nhắn hiện tại")
    history: Optional[str] = Field(default=None, description="Lịch sử chat")
    content_info: ContentInfo = Field(default_factory=ContentInfo)
    content_ideas: List[ContentIdea] = Field(default_factory=list, description="Ý tưởng nội dung")
    is_complete: bool = Field(default=False, description="Trạng thái hoàn thành")
    current_step: Optional[str] = Field(default="start", description="Bước hiện tại trong quy trình")
    def json_serializable(self):
        data = self.model_dump()
        data["content_info"] = self.content_info.model_dump()
        return data

In [3]:
import os
import sys

# Đặt đường dẫn giả lập để tránh lỗi inspect.getfile(cls)
sys.modules["__main__"].__file__ = os.path.abspath("fake_script.py")

In [5]:
import uuid
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, ValidationError, Field
from crewai.flow import Flow, listen, start, persist, router, or_
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
import os
from langchain_openai import ChatOpenAI
from llm import togetherai_llm
from datetime import datetime

# --- CrewAI Agent và Task ---
@CrewBase
class SocialContentIdeaCrew:
    """Social Content Creator crew"""
    
    # Định nghĩa config files
    agents_config = "config/agents.yaml"
    tasks_config = "config/tasks.yaml"

    @agent
    def content_manager(self) -> Agent:
        return Agent(
            config=self.agents_config["content_manager"],
            llm=togetherai_llm,
            verbose=True,
            memory=True
        )

    @task
    def content_ideation_task(self) -> Task:
        return Task(
            config=self.tasks_config["content_ideation_task"],
            agent=self.content_manager(),
            output_pydantic=ContentIdeaList,
            Verbose=True
        )
        
    @crew
    def crew(self) -> Crew:
        """Creates the Content Creator Crew"""
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True
        )
@CrewBase
class ContentCreationCrew:
    """Social Content Creator crew"""
    pass
class ContentInfo(BaseModel):
    topic: Optional[str] = Field(default=None, description="Chủ đề của nội dung")
    communication_goal: Optional[str] = Field(default=None, description="Mục tiêu truyền thông")
    target_audience: Optional[str] = Field(default=None, description="Đối tượng mục tiêu")
    brand_information: Optional[str] = Field(default=None, description="Thông tin thương hiệu")
    style_and_tone: Optional[str] = Field(default=None, description="Phong cách và giọng điệu")
# Define ContentIdea model
class ContentIdea(BaseModel):
    title: Optional[str] = Field(description="Tiêu đề ý tưởng")
    description: Optional[str] = Field(description="Mô tả ngắn gọn về ý tưởng")
    content_type: Optional[str] = Field(description="Loại nội dung (bài viết, hình ảnh, video, v.v.)")
    reason: Optional[str] = Field(description="Lý do ý tưởng phù hợp với mục tiêu và đối tượng")

# Create a wrapper class for the List[ContentIdea]
class ContentIdeaList(BaseModel):
    ideas: List[ContentIdea] = Field(default_factory=list)

class ChatState(BaseModel):
    id: str = ""
    message: Optional[str] =  Field(default=None, description="Nội dung tin nhắn hiện tại")
    history: Optional[str] = Field(default=None, description="Lịch sử chat")
    content_info: Dict[str, Any] = Field(default_factory=dict)
    content_ideas: List[ContentIdea] = Field(default_factory=list, description="Ý tưởng nội dung")
    is_complete: bool = Field(default=False, description="Trạng thái hoàn thành")
    current_step: Optional[str] = Field(default="start", description="Bước hiện tại trong quy trình")
    def model_dump(self, *args, **kwargs):
        # Chuyển đổi content_info thành dictionary trước khi dump
        data = super().model_dump(*args, **kwargs)
        if isinstance(self.content_info, ContentInfo):
            data["content_info"] = self.content_info.model_dump()
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> "ChatState":
        # Khôi phục lại content_info thành một object ContentInfo khi load
        if "content_info" in data and isinstance(data["content_info"], dict):
            data["content_info"] = ContentInfo(**data["content_info"])
        return cls(**data)

@persist()
class ContentChatFlow(Flow[ChatState]):
    def __init__(self, inputs=None, persistence=None):
        super().__init__(persistence=persistence)
        self.inputs = inputs or {}

        # Nếu trạng thái đã tồn tại, khôi phục lại content_info thành object ContentInfo
        if isinstance(self.state.content_info, dict):
            self.state.content_info = ContentInfo(**self.state.content_info)
        
        self.content_idea_crew = SocialContentIdeaCrew().crew()
        self.current_date = datetime.now().strftime("%Y-%m-%d-%H-%M")

        print(f"DEBUG: content_info before init: {self.inputs.get('content_info')}")
        print(f"DEBUG: Restored content_info: {self.state.content_info}")

    @start()
    def start_flow(self):
        print("DEBUG: Starting start_flow")
        print(f"INFO: Current state: {self.state.model_dump()}")
        print("Bắt đầu quy trình tạo nội dung")
        print("Thông tin nội dung:", self.state.content_info)
        self.state.current_step = "ideation"
        print(f"INFO: Updated state after start_flow: {self.state.model_dump()}")
        response = "Tôi sẽ giúp bạn tạo nội dung dựa trên thông tin bạn đã cung cấp. Đang bắt đầu quá trình tạo ý tưởng..."
        return response
    
    @listen(start_flow)
    def generate_content_ideas(self):
        print("DEBUG: Starting generate_content_ideas")
        #print(f"INFO: Current state before crew kickoff: {self.state.model_dump()}")
        print("Đang tạo ý tưởng nội dung...")
        inputs = {
            "message": self.state.message,
            "content_info": self.state.content_info,
            "history": self.state.history,
            "current_date": self.current_date
        }
        print(f"INFO: Inputs for crew: {inputs}")
        result =  self.content_idea_crew.kickoff(inputs=inputs)
        print(f"DEBUG: result.pydantic type: {type(result.pydantic)}")
        print(f"DEBUG: result.pydantic value: {result.pydantic}")  
        return result.raw
        # if hasattr(result, 'pydantic'):
        #     if isinstance(result.pydantic, ContentIdeaList):
        #         # Chuyển danh sách ContentIdea thành danh sách dictionary trước khi lưu vào state
        #         self.state.content_ideas = [idea.model_dump() for idea in result.pydantic.ideas]
                
        #     else:
        #         print("DEBUG: result.pydantic không phải ContentIdeaList")
        #         return "failed"
        #     ideas_text = ""
        #     for i, idea in enumerate(self.state.content_ideas):
        #         ideas_text += f"\n\n{i+1}. **{idea['title']}**\n"
        #         ideas_text += f"   - Loại nội dung: {idea['content_type']}\n"
        #         ideas_text += f"   - Mô tả: {idea['description']}\n"
        #         ideas_text += f"   - Lý do: {idea['reason']}"
                
        #     response = f"Tôi đã tạo ý tưởng nội dung dựa trên thông tin bạn cung cấp:{ideas_text}\n\n"
        #     return response
        # else:
        #     print("Không thể tạo ý tưởng nội dung")
        #     return "failed"
    

    # @listen(generate_content_ideas)
    # def start_research(self):
    #     pass
    # @listen(start_research)
    # def start_writing(self):
    #     pass
def run_chat_flow():
    chat_flow = ContentChatFlow()
    chat_id = str(uuid.uuid4())
    message = "Tôi muốn tạo nội dung về công nghệ cho doanh nghiệp"
    history = []
    content_info = ContentInfo(
        topic="Công nghệ",
        communication_goal="Quảng bá sản phẩm",
        target_audience="Doanh nghiệp",
        brand_information="Tên công ty: ABC Corp",
        style_and_tone="Chuyên nghiệp"
    )
    inputs={
                "id": chat_id,
                "message": message,
                "history": (
                    "\n".join(f"{msg['role']}: {msg['content']}" for msg in history)
                    if history
                    else ""
                ),
                "content_info": content_info.model_dump()
            }
    result = chat_flow.kickoff(
        inputs=inputs
    )
    print(result) 

In [6]:
run_chat_flow ()


DEBUG: content_info before init: None
DEBUG: Restored content_info: topic=None communication_goal=None target_audience=None brand_information=None style_and_tone=None
 No flow state found for UUID: d9c81806-0eb1-490d-a1ac-eabc92d19530
 Flow started with ID: d9c81806-0eb1-490d-a1ac-eabc92d19530


RuntimeError: asyncio.run() cannot be called from a running event loop